In [ ]:
!nvidia-smi

Thu Oct  7 08:47:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

ddata = pd.read_csv("drive/MyDrive/data/ddata.txt", encoding="euc-kr", sep="\t", names=["num","speaker","contents","tags"])
mdata = pd.read_csv("drive/MyDrive/data/mdata.txt", encoding="euc-kr", sep="\t", names=["id","part","category","exptype","exp"])

# 1. 전체 대화셋 개수 카운트

In [ ]:
dial_num = 0

for i in range(1, len(ddata)):
    if ddata.loc[i, "num"] == 0:
        dial_num += 1
print("총 대화셋 수: ",dial_num)

총 대화셋 수:  7235


# 2. Mdata 합치기

In [ ]:
c_mdata = pd.DataFrame({"id":[0],"exp":[None]})
c_mdata.drop(0, axis=0, inplace=True)
c_mdata

,id,exp


In [ ]:
id = ""
idx = 0
for i in range(len(mdata)):
    if id == mdata.loc[i, "id"]:
        c_mdata.loc[c_mdata.index[(c_mdata["id"] == id)].tolist().pop(),"exp"] = c_mdata.loc[c_mdata.index[(c_mdata["id"] == id)].tolist().pop(),"exp"] + " " + mdata.loc[i, "exp"]

    
    else:
        id = mdata.loc[i, "id"]
        c_mdata.loc[i, "id"] = id
        c_mdata.loc[i, "exp"] = mdata.loc[i, "exp"]

c_mdata.head()

,id,exp
0,BL-001,단추 여밈의 전체 오픈형 스탠드 칼라와 브이넥 네크라인의 결합스타일 손목까지 내려오...
20,BL-002,넓은 셔츠칼라 네크라인 앞중심에 반 오픈 끈여밈있는 스타일 드롭숄더 퍼프형 소매로 ...
38,BL-003,앞 중심에 단추 여밈 칼라가 없는 넓고 깊게 파인 브이넥 엉덩이 밑으로 내려오는 긴...
63,BL-004,"앞중심에 단추 여밈 칼라가 없는 라운드넥으로 골반 위에서 끝나는 짧은 기장 손목, ..."
84,BL-005,앞단추 여밈의 전체 오픈형 칼라가 없는 라운드넥크라인 엉덩이 위에서 끝나는 짧은 기...


In [ ]:
for i in range(len(c_mdata)): # id 뒤에 공백 없애기
    c_mdata.iloc[i].id = c_mdata.iloc[i].id.strip()
print(c_mdata.iloc[0].id)
c_mdata.reset_index(drop=True,inplace=True)
c_mdata.head()

BL-001


,id,exp
0,BL-001,단추 여밈의 전체 오픈형 스탠드 칼라와 브이넥 네크라인의 결합스타일 손목까지 내려오...
1,BL-002,넓은 셔츠칼라 네크라인 앞중심에 반 오픈 끈여밈있는 스타일 드롭숄더 퍼프형 소매로 ...
2,BL-003,앞 중심에 단추 여밈 칼라가 없는 넓고 깊게 파인 브이넥 엉덩이 밑으로 내려오는 긴...
3,BL-004,"앞중심에 단추 여밈 칼라가 없는 라운드넥으로 골반 위에서 끝나는 짧은 기장 손목, ..."
4,BL-005,앞단추 여밈의 전체 오픈형 칼라가 없는 라운드넥크라인 엉덩이 위에서 끝나는 짧은 기...


In [ ]:
print(len(c_mdata))

2603


In [ ]:
ddata.drop(ddata.index[ddata.tags == "INTRO"].tolist(), axis=0, inplace=True)
ddata[ddata.tags == "INTRO"]

,num,speaker,contents,tags


In [ ]:
ddata.fillna("", inplace=True)
ddata.reset_index(drop=True ,inplace=True)
ddata.head()

,num,speaker,contents,tags
0,1,<US>,처음 대학교 들어가는데 입을 옷 코디해주세요.,
1,2,<CO>,신입생 코디에 어울리게 화사한 스웨터를 추천해드릴게요.,EXP_RES_SITUATION;EXP_RES_DESCRIPTION
2,3,<AC>,SW-009,
3,4,<US>,이 옷에 어울리는 치마로 추천해주세요.,USER_SUCCESS
4,5,<AC>,SK-016,


# 3. NaN, Null 값 확인

In [ ]:
ddata.isnull().sum()

num         0
speaker     0
contents    0
tags        0
dtype: int64

In [ ]:
ddata.isna().sum()

num         0
speaker     0
contents    0
tags        0
dtype: int64

In [ ]:
ddata.isna().sum()

num         0
speaker     0
contents    0
tags        0
dtype: int64

In [ ]:
mdata.isnull().sum()

id          0
part        0
category    0
exptype     0
exp         0
dtype: int64

In [ ]:
c_mdata.isnull().sum()

id     0
exp    0
dtype: int64

# 4. 추천 성공 의상 아이디 종류 및 빈도 카운트 

In [ ]:
c_data = pd.DataFrame({"contents":[None], "id":["xx-xxx"]})
c_data.drop(0,axis=0,inplace=True)
c_data.head()

,contents,id


In [ ]:
ddata.head()

,num,speaker,contents,tags
0,1,<US>,처음 대학교 들어가는데 입을 옷 코디해주세요.,
1,2,<CO>,신입생 코디에 어울리게 화사한 스웨터를 추천해드릴게요.,EXP_RES_SITUATION;EXP_RES_DESCRIPTION
2,3,<AC>,SW-009,
3,4,<US>,이 옷에 어울리는 치마로 추천해주세요.,USER_SUCCESS
4,5,<AC>,SK-016,


In [ ]:
dial = ddata.loc[0, "contents"].strip()
ac = ""

for i in range(1,len(ddata)):
    cur_sen = ddata.loc[i,"contents"].strip()

    if ddata.loc[i, "num"] > ddata.loc[i-1, "num"]: # 이전 대화셋에 포함된 문장인지 확인
        
        if ddata.loc[i,"speaker"] == "<AC>": # case 1. 의상 아이디가 나온 경우
            if len(ac) > 1:
                ac = ac + " " + cur_sen # 의상 아이디 저장
            else:
                ac = cur_sen

        elif "USER_SUCCESS" == ddata.loc[i,"tags"]: # case 2. USER_SUCCESS 태그가 나온 경우
            if 0 < len(ac) < 7: # 의상을 하나만 추천한 경우
                dial = dial + " " + ac + " " + cur_sen
                c_data.loc[i, "contents"] =  dial + " " + c_mdata.loc[c_mdata.id == ac].exp.values[0]
                c_data.loc[i, "id"] = ac
                ac = ""

            elif len(ac) == 0:
                dial = dial + " " + cur_sen

            else: # 의상을 한 번에 여러 개 추천한 경우
                idx_cnt = 0
                for one_ac in ac.split():
                    c_data.loc[i + idx_cnt, "contents"] =  dial + " " + one_ac + " "+ cur_sen + " " + c_mdata.loc[c_mdata.id == one_ac].exp.values[0]
                    c_data.loc[i + idx_cnt, "id"] = one_ac
                    idx_cnt += 1
                dial = dial + " " + ac + " " + cur_sen    
                ac = ""
        
        elif "USER_FAIL" in ddata.loc[i,"tags"]:
            dial = dial + " " + ac + " " + cur_sen
            ac = ""

        else: # case 3. 그냥 대화문인 경우
            dial = dial + " " + cur_sen
    
    else: #새로운 대화셋 시작이면 초기화
        dial = cur_sen
        ac = ""


In [ ]:
print("전체 추천 성공 횟수: ", len(c_data))
c_data.head()

전체 추천 성공 횟수:  34303


,contents,id
3,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,SW-009
12,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,SK-053
17,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,CT-019
22,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,SE-039
26,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,CT-019


In [ ]:
c_data.reset_index(drop=True ,inplace=True)
print("성공한 의상 종류의 수: ", c_data.id.nunique())
c_data.head()

성공한 의상 종류의 수:  2316


,contents,id
0,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,SW-009
1,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,SK-053
2,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,CT-019
3,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,SE-039
4,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,CT-019


In [ ]:
c_data.loc[0].contents

'처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터를 추천해드릴게요. SW-009 이 옷에 어울리는 치마로 추천해주세요. 라운드 네크라인 여밈이 없는 풀오버 스타일 전체 케이블 조직 드롭숄더 슬리브 손등까지 오는 길이의 소매 몸에 적당히 맞는 품 엉덩이를 살짝 덮는 길이 네크라인, 소매 끝, 밑단에 고무단 처리 레이온52%,나일론20%,폴리에스터28% 꽈배기 조직, 플레인 조직이 섞여 있는 형태 드라이 크리닝 단색의 라임색 브라이트톤(고명도 고채도)의 옐로우 그린 산뜻하고 발랄한 느낌 기본의 단정한 편안한 산뜻한 발랄한 경쾌한 따뜻한 데일리룩 캠퍼스룩'

In [ ]:
for i in range(len(c_data)):
    c_data["id"][i] = c_data["id"][i][0:2]

c_data.reset_index(drop=True, inplace=True)
print("데이터 개수: ",len(c_data), ", 의상 종류 수: ", c_data.id.nunique())
c_data.head()

데이터 개수:  34303 , 의상 종류 수:  13


,contents,id
0,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,SW
1,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,SK
2,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,CT
3,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,SE
4,처음 대학교 들어가는데 입을 옷 코디해주세요. 신입생 코디에 어울리게 화사한 스웨터...,CT


In [ ]:
c_data["contents"] = c_data["contents"].str.replace("[^A-Za-z0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
c_data.head()

,contents,id
0,처음 대학교 들어가는데 입을 옷 코디해주세요 신입생 코디에 어울리게 화사한 스웨터를...,SW
1,처음 대학교 들어가는데 입을 옷 코디해주세요 신입생 코디에 어울리게 화사한 스웨터를...,SK
2,처음 대학교 들어가는데 입을 옷 코디해주세요 신입생 코디에 어울리게 화사한 스웨터를...,CT
3,처음 대학교 들어가는데 입을 옷 코디해주세요 신입생 코디에 어울리게 화사한 스웨터를...,SE
4,처음 대학교 들어가는데 입을 옷 코디해주세요 신입생 코디에 어울리게 화사한 스웨터를...,CT


In [ ]:
c_data.id.value_counts()

SE    8864
PT    5568
KN    2957
SK    2500
CT    2232
BL    1970
JK    1966
JP    1826
OP    1745
SW    1731
CD    1466
SH    1206
VT     272
Name: id, dtype: int64

In [ ]:
c_data.id.unique().tolist()

['SW', 'SK', 'CT', 'SE', 'PT', 'JP', 'CD', 'KN', 'OP', 'BL', 'SH', 'JK', 'VT']

In [ ]:
c_data.loc[c_data["id"] == "SW", "id"] = 0
c_data.loc[c_data["id"] == "SK", "id"] = 1
c_data.loc[c_data["id"] == "CT", "id"] = 2
c_data.loc[c_data["id"] == "SE", "id"] = 3
c_data.loc[c_data["id"] == "PT", "id"] = 4
c_data.loc[c_data["id"] == "JP", "id"] = 5
c_data.loc[c_data["id"] == "CD", "id"] = 6
c_data.loc[c_data["id"] == "KN", "id"] = 7
c_data.loc[c_data["id"] == "OP", "id"] = 8
c_data.loc[c_data["id"] == "BL", "id"] = 9
c_data.loc[c_data["id"] == "SH", "id"] = 10
c_data.loc[c_data["id"] == "JK", "id"] = 11
c_data.loc[c_data["id"] == "VT", "id"] = 12

In [ ]:
c_data.head()

,contents,id
0,처음 대학교 들어가는데 입을 옷 코디해주세요 신입생 코디에 어울리게 화사한 스웨터를...,0
1,처음 대학교 들어가는데 입을 옷 코디해주세요 신입생 코디에 어울리게 화사한 스웨터를...,1
2,처음 대학교 들어가는데 입을 옷 코디해주세요 신입생 코디에 어울리게 화사한 스웨터를...,2
3,처음 대학교 들어가는데 입을 옷 코디해주세요 신입생 코디에 어울리게 화사한 스웨터를...,3
4,처음 대학교 들어가는데 입을 옷 코디해주세요 신입생 코디에 어울리게 화사한 스웨터를...,2


# 5. Bert 돌리기 

In [ ]:
from sklearn.model_selection import train_test_split

train_idx, test_idx,_ ,_ = train_test_split(c_data.index, c_data.id, test_size=0.2)
train_set = c_data.iloc[train_idx]
test_set = c_data.iloc[test_idx]
train_set.reset_index(drop=True, inplace=True)

In [ ]:
train_idx, valid_idx, _, _ = train_test_split(train_set.index, train_set.id, test_size=0.2)
valid_set = train_set.iloc[valid_idx]
train_set = train_set.iloc[train_idx]

In [ ]:
train_set.shape, valid_set.shape, test_set.shape

((21953, 2), (5489, 2), (6861, 2))

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 36.5 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 3.3 MB 35.0 MB/s 
     |████████████████████████████████| 636 kB 41.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from transformers import BertTokenizerFast

class BertDataset(torch.utils.data.Dataset):

    def __init__(self, contents, id, tokenizer):
        self.contents    = contents
        self.id = id
        self.tokenizer  = tokenizer
        self.max_len    = tokenizer.model_max_length
  
    def __len__(self):
        return len(self.contents)
  
    def __getitem__(self, index):
        contents = str(self.contents[index])
        id = self.id[index]

        encoded_contents = self.tokenizer.encode_plus(
            contents,
            add_special_tokens    = True,
            max_length            = self.max_len,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors        = "pt",
            padding               = "max_length",
            truncation            = True
        )

        return {
            'input_ids': encoded_contents['input_ids'][0],
            'attention_mask': encoded_contents['attention_mask'][0],
            'labels': torch.tensor(id, dtype=torch.long)
        }

In [ ]:
bert_model_name = 'beomi/kcbert-large'

tokenizer = BertTokenizerFast.from_pretrained(bert_model_name)

Downloading:   0%|          | 0.00/244k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/672 [00:00<?, ?B/s]

In [ ]:
train_set_dataset = BertDataset(
    contents    = train_set.contents.tolist(),
    id = train_set.id.tolist(),
    tokenizer  = tokenizer,
)

valid_set_dataset = BertDataset(
    contents    = valid_set.contents.tolist(),
    id = valid_set.id.tolist(),
    tokenizer  = tokenizer,
)

train_set_dataloader = torch.utils.data.DataLoader(
    train_set_dataset,
    batch_size  = 16,
    num_workers = 4
)

valid_set_dataloader = torch.utils.data.DataLoader(
    valid_set_dataset,
    batch_size  = 16,
    num_workers = 4
)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
from transformers import BertForSequenceClassification

num_labels = 13
model = BertForSequenceClassification.from_pretrained(bert_model_name, num_labels=num_labels)

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initial

In [ ]:
tl_strategy = 3

if tl_strategy == 1:
  for name, param in model.bert.named_parameters():
    param.requires_grad = False

elif tl_strategy == 2:
  for name, param in model.bert.named_parameters():
    if not name.startswith('pooler'):
      param.requires_grad = False

elif tl_strategy == 3:
  for name, param in model.bert.named_parameters():
    if ( not name.startswith('pooler') ) and "layer.23" not in name :
      param.requires_grad = False

In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_dir = 'drive/MyDrive'

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir                  = model_dir,
    num_train_epochs            = 2,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 8, 
    warmup_steps                = 50,
    weight_decay                = 0.01,
    save_strategy               = "epoch",
    evaluation_strategy         = "steps"
)

trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_set_dataset,
    eval_dataset    = valid_set_dataset,
    compute_metrics = compute_metrics
)

In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_dir = './model'

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir                  = model_dir,
    num_train_epochs            = 2,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 8, 
    warmup_steps                = 50,
    weight_decay                = 0.01,
    save_strategy               = "epoch",
    evaluation_strategy         = "steps"
)

trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_set_dataset,
    eval_dataset    = valid_set_dataset,
    compute_metrics = compute_metrics
)

trainer.train()

***** Running training *****
  Num examples = 21953
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2746


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.834200,0.086731,0.978320,[0.97426471 0.98429319 0.97101449 0.97674419 0.98483207 0.98611111 0.96344086 0.98367791 0.97363796 0.98555377 0.96453901 0.97902098 0.92063492],[0.96715328 0.98687664 0.98240469 0.95903291 0.99344262 0.99300699 0.99115044 0.98689956 0.96515679 0.98713826 0.96682464 0.99644128 1. ],[0.98148148 0.98172324 0.95988539 0.99512195 0.9763695 0.97931034 0.93723849 0.98047722 0.9822695 0.98397436 0.96226415 0.96219931 0.85294118]
1000,0.107000,0.059363,0.985061,[0.99065421 0.98691099 0.97965116 0.98243197 0.98927039 0.99481865 0.98297872 0.98908297 0.98586572 0.98726115 0.97607656 0.97916667 0.92957746],[1. 0.98950131 0.99410029 0.97138965 0.98821008 0.99653979 1. 0.9956044 0.98239437 0.98101266 0.99029126 0.98947368 0.89189189],[0.98148148 0.9843342 0.96561605 0.99372822 0.99033298 0.99310345 0.9665272 0.98264642 0.9893617 0.99358974 0.96226415 0.96907216 0.97058824]


***** Running Evaluation *****
  Num examples = 5489
  Batch size = 8
Trainer is attempting to log a value of "[0.97426471 0.98429319 0.97101449 0.97674419 0.98483207 0.98611111
 0.96344086 0.98367791 0.97363796 0.98555377 0.96453901 0.97902098
 0.92063492]" of type <class 'numpy.ndarray'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.96715328 0.98687664 0.98240469 0.95903291 0.99344262 0.99300699
 0.99115044 0.98689956 0.96515679 0.98713826 0.96682464 0.99644128
 1.        ]" of type <class 'numpy.ndarray'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.98148148 0.98172324 0.95988539 0.99512195 0.9763695  0.97931034
 0.93723849 0.98047722 0.9822695  0.98397436 0.96226415 0.96219931
 0.85294118]" of type <class 'numpy.ndarray'> for ke

TypeError: ignored

In [ ]:
import os
model = BertForSequenceClassification.from_pretrained(os.path.join(model_dir, 'checkpoint-1373'))

test_set_dataset = BertDataset(
    contents = test_set.contents.tolist(),
    id = test_set.id.tolist(),
    tokenizer  = tokenizer,
)

training_args = TrainingArguments(
    output_dir = model_dir,
    do_predict = True
)

trainer = Trainer(
    model           = model,
    args            = training_args,
    compute_metrics = compute_metrics,
)

trainer.predict(test_set_dataset)

loading configuration file drive/MyDrive/checkpoint-1373/config.json
Model config BertConfig {
  "_name_or_path": "beomi/kcbert-large",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps": 1e-1

PredictionOutput(predictions=array([[-1.1483195e+00, -4.1575643e-01, -1.4015057e+00, ...,
        -1.4498503e+00, -2.4574068e+00, -1.9820623e-01],
       [ 6.7559075e-01, -2.8559320e+00, -1.5455655e+00, ...,
         1.1150545e+01,  1.2652997e+00, -7.1594313e-02],
       [-1.1362808e+00, -1.7866157e+00, -1.2374905e+00, ...,
        -2.2405994e+00, -2.1565263e+00, -3.0033847e-02],
       ...,
       [-2.1510537e+00,  1.1013646e+01, -1.0244130e+00, ...,
         2.5821415e-01,  8.9383912e-01,  3.5400540e-01],
       [-1.7383542e+00, -1.7342710e+00,  9.3176281e-03, ...,
        -5.7321233e-01, -9.8232049e-01, -6.6839385e-01],
       [-5.5845392e-01, -1.2503967e+00, -1.9085951e+00, ...,
        -7.8304535e-01, -3.0413668e+00,  3.9533293e-01]], dtype=float32), label_ids=array([ 4, 10,  4, ...,  1,  3,  4]), metrics={'test_loss': 0.0494103878736496, 'test_accuracy': 0.9870281300102026, 'test_f1': array([0.98795181, 0.99310345, 0.99141631, 0.99028571, 0.98783236,
       0.98637602, 0.97605893